In [2]:
import pandas as pd
import numpy as np
import requests
import socket
import json
import time
import logging
import random
import sys
import asyncio
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s [%(levelname)s] %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)


## Interface

加入了并发的代码

In [87]:
class InterfaceClass:
    def __init__(self, domain_name):
        self.domain_name = domain_name
        self.session = requests.Session()
    
    def send_request(self, method, url, data):
        try:
            response = self.session.post(url, data=json.dumps(data)).json()
            return response
        except requests.RequestException as e:
            logger.error(f"Request failed: {e}")
            return None
    
    def sendLogin(self, username, password):
        url = self.domain_name + "/Login"
        data = {
            "user": username,
            "password": password
        }
        return self.send_request("POST", url, data)

    def sendOrder(self, token_ub, instrument, localtime, direction, price, volume):
        logger.debug("Order: Instrument: {}, Direction:{}, Price: {}, Volume:{}".format(instrument, direction, price, volume))
        url = self.domain_name + "/TradeAPI/Order"
        data = {
            "token_ub": token_ub,
            "user_info": "NULL",
            "instrument": instrument,
            "localtime": localtime,
            "direction": direction,
            "price": price,
            "volume": volume,
        }
        print(data)
        return self.send_request("POST", url, data)
    
    def sendCancel(self, token_ub, instrument, localtime, index):
        logger.debug("Cancel: Instrument: {}, index:{}".format(instrument, index))
        url = self.domain_name + "/TradeAPI/Cancel"
        data = {
            "token_ub": token_ub,
            "user_info": "NULL",
            "instrument": instrument,
            "localtime": 0,
            "index": index
        }
        return self.send_request("POST", url, data)
        
    def sendGetLimitOrderBook(self, token_ub, instrument):
        logger.debug("GetLimitOrderBOok: Instrument: {}".format(instrument))
        url = self.domain_name + "/TradeAPI/GetLimitOrderBook"
        data = {
            "token_ub": token_ub,
            "instrument": instrument
        }
        return self.send_request("POST", url, data)
    
    def sendGetAllLimitOrderBooks(self, token_ub):
        # logger.debug("GetAllLimitOrderBooks")
        url = self.domain_name + "/TradeAPI/GetAllLimitOrderBooks"
        data = {
            "token_ub": token_ub
        }
        return self.send_request("POST", url, data)
    
    def sendGetUserInfo(self, token_ub):
        logger.debug("GetUserInfo: ")
        url = self.domain_name + "/TradeAPI/GetUserInfo"
        data = {
            "token_ub": token_ub,
        }
        return self.send_request("POST", url, data)

    def sendGetGameInfo(self, token_ub):
        logger.debug("GetGameInfo: ")
        url = self.domain_name + "/TradeAPI/GetGameInfo"
        data = {
            "token_ub": token_ub,
        }
        return self.send_request("POST", url, data)

    def sendGetInstrumentInfo(self, token_ub):
        logger.debug("GetInstrumentInfo: ")
        url = self.domain_name + "/TradeAPI/GetInstrumentInfo"
        data = {
            "token_ub": token_ub,
        }
        return self.send_request("POST", url, data)

    def sendGetTrade(self, token_ub, instrument):
        logger.debug("GetTrade: Instrment: {}".format(instrument))
        url = self.domain_name + "/TradeAPI/GetTrade"
        data = {
            "token_ub": token_ub,
            "instrument_name": instrument
        }
        return self.send_request("POST", url, data)
        
    def sendGetAllTrades(self, token_ub):
        logger.debug("GetAllTrades: ")
        url = self.domain_name + "/TradeAPI/GetAllTrades"
        data = {
            "token_ub": token_ub,
        }
        return self.send_request("POST", url, data)
    
    def sendGetActiveOrder(self, token_ub):
        logger.debug("GetActiveOrder: ")
        url = self.domain_name + "/TradeAPI/GetActiveOrder"
        data = {
            "token_ub": token_ub,
        }
        return self.send_request("POST", url, data)

    def sendRequestsConcurrently(self, request_function, *args_list, max_workers=55):
        results = []
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = [executor.submit(request_function, *args) for args in args_list]
            for future in as_completed(futures):
                result = future.result()
                results.append(result)
        return results

# BotsClass

In [5]:
class BotsClass:
    def __init__(self, username, password):
        self.username = username
        self.password = password
    def login(self):
        pass
    def init(self):
        pass
    def bod(self):
        pass
    def work(self):
        pass
    def eod(self):
        pass
    def final(self):
        pass

class BotsDemoClass(BotsClass):
    def __init__(self, username, password, port):
        super().__init__(username, password)
        self.api = InterfaceClass(f"http://8.147.116.35:{port}")
    
    def login(self):
        response = self.api.sendLogin(self.username, self.password)
        if response["status"] == "Success":
            self.token_ub = response["token_ub"]
            logger.info("Login Success: {}".format(self.token_ub))
        else:
            print('\033[31mlogin error \033[0m',response)
            logger.info("Login Error: ", response)
    
    def GetInstruments(self):
        response = self.api.sendGetInstrumentInfo(self.token_ub)
        print(response)
        if response["status"] == "Success":
            self.instruments = []
            for instrument in response["instruments"]:
                self.instruments.append(instrument["instrument_name"])
            logger.info("Get Instruments: {}".format(self.instruments))
    
    def init(self):
        response = self.api.sendGetGameInfo(self.token_ub)
        if response["status"] == "Success":
            self.start_time = response["next_game_start_time"]
            self.running_days = response["next_game_running_days"]
            self.running_time = response["next_game_running_time"]
            self.time_ratio = response["next_game_time_ratio"]
        print(response)
        
        self.GetInstruments()
        self.day = 0
        
    def bod(self):
        pass        
    
    def work(self): 
        userinfo = self.api.sendGetUserInfo(self.token_ub)
        stockID = random.randint(0, len(self.instruments) - 1)
        # print(stockID, self.instruments[stockID])
        LOB = self.api.sendGetLimitOrderBook(self.token_ub, self.instruments[stockID])
        if LOB["status"] == "Success":
            askprice_1 = float(LOB["lob"]["askprice"][0])
            bidprice_1 = float(LOB["lob"]["bidprice"][0])
            t = ConvertToSimTime_us(self.start_time, self.time_ratio, self.day, self.running_time)
            #target volume
            if userinfo["rows"][stockID]["target_volume"] > 0:
                response = self.api.sendOrder(self.token_ub, self.instruments[stockID], t, "buy", askprice_1, 100)
            else:
                response = self.api.sendOrder(self.token_ub, self.instruments[stockID], t, "sell", bidprice_1, 100)
            if response["index"] is not None:
                resp = self.api.sendCancel(self.token_ub, self.instruments[stockID], t, response["index"])
                print(resp)
            else:
                print('index none')
    def eod(self):
        pass
    def final(self):
        pass

## FastBot

In [6]:

class FastBot(BotsDemoClass):
    def __init__(self, username, password, port, spread_threshold=0.05, max_position=1000):
        super().__init__(username, password, port)
        self.spread_threshold = spread_threshold
        self.max_position = max_position
        self.position = 0
    
    def work(self): 
        # Get user info and select a random instrument
        userinfo = self.api.sendGetUserInfo(self.token_ub)
        stockID = random.randint(0, len(self.instruments) - 1)
        instrument = self.instruments[stockID]
        
        # Get the Limit Order Book (LOB)
        LOB = self.api.sendGetLimitOrderBook(self.token_ub, instrument)
        if LOB["status"] == "Success":
            askprice_1 = float(LOB["lob"]["askprice"][0])
            bidprice_1 = float(LOB["lob"]["bidprice"][0])
            
            # Simple optimization based on spread
            spread = askprice_1 - bidprice_1
            if spread >= self.spread_threshold:
                if self.position < self.max_position:
                    # Place buy order
                    response = self.api.sendOrder(self.token_ub, instrument, 
                                                  time.time(), "buy", bidprice_1, 100)
                    if response["status"] == "Success":
                        self.position += 100
                        logging.info(f"Bought 100 units at {bidprice_1}. New position: {self.position}")
                
                if self.position > 0:
                    # Place sell order
                    response = self.api.sendOrder(self.token_ub, instrument, 
                                                  time.time(), "sell", askprice_1, 100)
                    if response["status"] == "Success":
                        self.position -= 100
                        logging.info(f"Sold 100 units at {askprice_1}. New position: {self.position}")
            else:
                logging.info(f"Spread {spread} is below threshold {self.spread_threshold}, no trade made.")
    
    def run(self):
        self.login()
        self.init()
        SimTimeLen = 3000 # 60*5 * 10
        endWaitTime = 600 # 60*1 * 10
        while ConvertToSimTime_us(self.start_time, self.time_ratio, self.day, self.running_time) >= SimTimeLen:
            self.day += 1

        while self.day <= self.running_days:
            while ConvertToSimTime_us(self.start_time, self.time_ratio, self.day, self.running_time) <= -900:
                pass
            
            self.bod()
            now = round(ConvertToSimTime_us(self.start_time, self.time_ratio, self.day, self.running_time))
            for s in range(now, SimTimeLen + endWaitTime):
                while ConvertToSimTime_us(self.start_time, self.time_ratio, self.day, self.running_time) < s:
                    pass
                
                start_time = time.time()
                self.work()
                end_time = time.time()
                time_to_wait = 0.1 - (end_time - start_time)
                if time_to_wait > 0:
                    time.sleep(time_to_wait)
            
            self.eod()
            self.day += 1
        
        self.final()

## 做市商策略

In [7]:
class BotsMarketMaker(BotsClass):
    def __init__(self, username, password, port):
        super().__init__(username, password)
        self.api = InterfaceClass(f"http://8.147.116.35:{port}")
        self.token_ub = None
        self.target_position = None  # 目标仓位
        self.current_position = 0  # 当前持仓
        self.pending_orders = []  # 未成交订单列表
        self.day = 1
        self.running_days = 5  # 假设运行5天
        self.start_time = 0  # 假设初始化时刻为0
        self.time_ratio = 1  # 模拟时间比例
        self.running_time = 0  # 运行时间

    def login(self):
        # 登录到交易系统
        response = self.api.sendLogin(self.username, self.password)
        if response["status"] == "Success":
            self.token_ub = response["token_ub"]
            logger.info("Login Success: {}".format(self.token_ub))
        else:
            print('\033[31mlogin error \033[0m',response)
            logger.info("Login Error: ", response)
    
    def GetInstruments(self):
        response = self.api.sendGetInstrumentInfo(self.token_ub)
        print(response)
        if response["status"] == "Success":
            self.instruments = []
            for instrument in response["instruments"]:
                self.instruments.append(instrument["instrument_name"])
            logger.info("Get Instruments: {}".format(self.instruments))
    
    def getTargetPositions(self):
        response = self.api.sendGetUserInfo(self.token_ub)
        self.target_position = [row['target_volume'] for row in response['rows']]
    
    def init(self):
        response = self.api.sendGetGameInfo(self.token_ub)
        if response["status"] == "Success":
            self.start_time = response["next_game_start_time"]
            self.running_days = response["next_game_running_days"]
            self.running_time = response["next_game_running_time"]
            self.time_ratio = response["next_game_time_ratio"]
        print(response)
        
        self.GetInstruments()
        
        self.current_positions = {symbol: 0 for symbol in self.instruments}  # 当前持仓
        self.pending_orders = {symbol: [] for symbol in self.target_position}  
        self.day = 0

    def bod(self):
        # 每日开始时的操作，可以用于初始化当日的状态
        for symbol in self.pending_orders:
            self.pending_orders[symbol] = []
        logger.info(f"Day {self.day} started.")
    
    def work(self):
        # 核心交易逻辑
        current_time = ConvertToSimTime_us(self.start_time, self.time_ratio, self.day, self.running_time)
        if current_time >= self.end_time - 30:  # 临近收盘
            self.execute_trades()
    
    def execute_trades(self):
        # 根据当前的市场信息执行交易
        remaining_quantity = self.target_position - self.current_position
        
        # 获取当前未完成的订单量
        unfilled_orders = self.get_unfilled_order_quantity()
        allowed_quantity = remaining_quantity - unfilled_orders

        if allowed_quantity <= 0:
            return

        # 获取当前市场价格及挂单信息
        lob = self.api_interface.sendGetLimitOrderBook()
        best_bid = lob["best_bid"]
        best_ask = lob["best_ask"]

        if remaining_quantity > 0:
            # 买入操作 - 作为做市商，稍微低于市场的买入价挂单
            bid_price = best_bid - self.calculate_spread()
            self.place_order("buy", bid_price, min(allowed_quantity, remaining_quantity))
        elif remaining_quantity < 0:
            # 卖出操作 - 作为做市商，稍微高于市场的卖出价挂单
            ask_price = best_ask + self.calculate_spread()
            self.place_order("sell", ask_price, min(allowed_quantity, -remaining_quantity))
    
    def calculate_spread(self):
        # 计算做市商的挂单价差，可以根据市场波动性和流动性动态调整
        base_spread = 0.01  # 假设一个基本价差
        volatility_factor = random.uniform(0.5, 1.5)  # 根据市场波动性调整价差
        return base_spread * volatility_factor
    
    def place_order(self, direction, price, quantity):
        # 根据方向、价格和数量挂单
        token = self.api_interface.get_token()
        instrument = "instrument_name"  # 根据需要填写合适的交易工具名称
        localtime = ConvertToSimTime_us(self.start_time, self.time_ratio, self.day, self.running_time)
        
        response = self.api_interface.sendOrder(token, instrument, localtime, direction, price, quantity)
        if response["status"] == "success":
            self.pending_orders.append(response["order_id"])
            logger.info(f"Placed {direction} order: Price: {price}, Quantity: {quantity}")
    
    def get_unfilled_order_quantity(self):
        # 获取所有未完成订单的总量
        total_unfilled = 0
        for order_id in self.pending_orders:
            order_info = self.api_interface.sendGetActiveOrder(order_id)
            total_unfilled += order_info["unfilled_quantity"]
        return total_unfilled
    
    def eod(self):
        # 每日结束时的操作
        logger.info(f"Day {self.day} ended. Final Position: {self.current_position}")
        # 可以在此处撤销未成交订单，重新评估仓位等
    
    def final(self):
        # 结束所有运行时的操作
        logger.info("All days completed. Finalizing...")
        self.api_interface.logout()

# API测试

In [91]:
api =  InterfaceClass(f"http://8.147.116.35:{30020}")

In [92]:
username = 'UBIQ_TEAM179'
password = 'ANfgOwr3SvpN'

In [94]:
api.sendLogin(username, password)

{'response_type': 'login_response',
 'token_ub': 'UBIQ_TEAM179_ANfgOwr3SvpN',
 'status': 'Success'}

In [11]:
token_ub = 'UBIQ_TEAM179_ANfgOwr3SvpN'

In [73]:
user_info = api.sendGetUserInfo(token_ub)
uf_df = convert_userinfo_response_to_df_format(user_info['rows'], 1)

2024-08-28 16:10:47,378 [DEBUG] GetUserInfo: 


In [ ]:
{'token_ub': 'UBIQ_TEAM179_ANfgOwr3SvpN', 'user_info': '', 'instrument': 'UBIQ046', 'localtime': 67, 'direction': 'buy', 'price': 16.85, 'volume': 1100}

In [ ]:
{'token_ub': 'UBIQ_TEAM179_ANfgOwr3SvpN', 'user_info': '', 'instrument': 'UBIQ003', 'localtime': 2070.150730609894, 'index': 62476}

In [95]:
api.sendGetActiveOrder(token_ub)['instruments']

2024-08-28 16:49:48,072 [DEBUG] GetActiveOrder: 


[{'instrument_name': 'UBIQ000', 'active_orders': []},
 {'instrument_name': 'UBIQ001', 'active_orders': []},
 {'instrument_name': 'UBIQ002', 'active_orders': []},
 {'instrument_name': 'UBIQ003', 'active_orders': []},
 {'instrument_name': 'UBIQ004', 'active_orders': []},
 {'instrument_name': 'UBIQ005', 'active_orders': []},
 {'instrument_name': 'UBIQ006', 'active_orders': []},
 {'instrument_name': 'UBIQ007', 'active_orders': []},
 {'instrument_name': 'UBIQ008', 'active_orders': []},
 {'instrument_name': 'UBIQ009', 'active_orders': []},
 {'instrument_name': 'UBIQ010', 'active_orders': []},
 {'instrument_name': 'UBIQ011', 'active_orders': []},
 {'instrument_name': 'UBIQ012', 'active_orders': []},
 {'instrument_name': 'UBIQ013', 'active_orders': []},
 {'instrument_name': 'UBIQ014', 'active_orders': []},
 {'instrument_name': 'UBIQ015', 'active_orders': []},
 {'instrument_name': 'UBIQ016', 'active_orders': []},
 {'instrument_name': 'UBIQ017', 'active_orders': []},
 {'instrument_name': 'UBIQ01

In [96]:
api.sendCancel(token_ub, 'UBIQ045', 2525.788621902466, 22127)

2024-08-28 16:49:56,231 [DEBUG] Cancel: Instrument: UBIQ045, index:22127


{'response_type': 'cancel_response',
 'user_info': None,
 'localtime': 2443.390599999984,
 'status': 'Success'}

In [41]:
api.sendGetUserInfo(token_ub)

2024-08-28 14:56:35,316 [DEBUG] GetUserInfo: 


{'response_type': 'get_user_info_response',
 'orders': 575,
 'error_orders': 0,
 'order_value': 38113784,
 'trade_value': 38204230,
 'rows': [{'instrument_name': 'UBIQ000',
   'share_holding': 7800,
   'orders': 8,
   'error_orders': 0,
   'order_value': 398509,
   'trade_value': 394404,
   'target_volume': 7800,
   'remain_volume': 0,
   'frozen_volume': 0},
  {'instrument_name': 'UBIQ001',
   'share_holding': -14400,
   'orders': 18,
   'error_orders': 0,
   'order_value': 104116,
   'trade_value': 106903,
   'target_volume': -14400,
   'remain_volume': 0,
   'frozen_volume': 0},
  {'instrument_name': 'UBIQ002',
   'share_holding': -14100,
   'orders': 17,
   'error_orders': 0,
   'order_value': 4682138,
   'trade_value': 4731960,
   'target_volume': -14100,
   'remain_volume': 0,
   'frozen_volume': 0},
  {'instrument_name': 'UBIQ003',
   'share_holding': -8100,
   'orders': 7,
   'error_orders': 0,
   'order_value': 2019028,
   'trade_value': 2038963,
   'target_volume': -8100,
   

In [81]:
%%time
def execution_time(func, *argv):
    start_time = time.time()
    func(*argv)
    end_time = time.time()
    return (end_time - start_time) * 1000

CPU times: user 3 µs, sys: 4 µs, total: 7 µs
Wall time: 7.15 µs


In [82]:
time_list = []
for i in range(10):
    time_list.append(execution_time(api.sendGetAllLimitOrderBooks, token_ub))

In [85]:
time_list

[42.14215278625488,
 56.47397041320801,
 47.0278263092041,
 43.351173400878906,
 39.43490982055664,
 40.40980339050293,
 40.986061096191406,
 36.54789924621582,
 37.16111183166504,
 32.730817794799805]

In [162]:
api.sendOrder(token_ub, "UBIQ000", 1, 'buy', 49.83, 100)

2024-08-27 20:39:17,586 [DEBUG] Order: Instrument: UBIQ000, Direction:buy, Price: 49.83, Volume:100


{'token_ub': 'UBIQ_TEAM179_ANfgOwr3SvpN', 'user_info': 'NULL', 'instrument': 'UBIQ000', 'localtime': 1, 'direction': 'buy', 'price': 49.83, 'volume': 100}


{'response_type': 'order_response',
 'user_info': None,
 'localtime': 1646.27496000001,
 'index': 112466,
 'status': 'Success'}

In [164]:
a = 0.4

In [173]:
temp = pd.read_csv('temp.csv')

In [181]:
temp.iloc[0].squeeze()['AskPrice1']

6.47

In [172]:
a['AskPrice1']

7.49

In [31]:
api.sendGetUserInfo(token_ub)['rows'][8]

2024-08-28 13:24:56,816 [DEBUG] GetUserInfo: 


{'instrument_name': 'UBIQ008',
 'share_holding': -5800,
 'orders': 13,
 'error_orders': 594,
 'order_value': 999703,
 'trade_value': 1010961,
 'target_volume': -6000,
 'remain_volume': -200,
 'frozen_volume': 0}

In [37]:
api.sendGetAllTrades(token_ub)

2024-08-28 13:48:55,298 [DEBUG] GetAllTrades: 


{'response_type': 'get_all_trade_response',
 'trade_lists': [[],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []],
 'status': 'Success'}

In [163]:
api.sendGetAllLimitOrderBooks(token_ub)['lobs'][0]

{'localtime': 1667,
 'limit_up_price': 62.09375000000001,
 'limit_down_price': 37.25625,
 'bidprice': [49.87,
  49.86,
  49.85,
  49.84,
  49.83,
  49.82,
  49.81,
  49.8,
  49.79,
  49.78],
 'askprice': [49.88,
  49.89,
  49.9,
  49.91,
  49.92,
  49.93,
  49.94,
  49.95,
  49.96,
  49.97],
 'bidvolume': [4300, 2200, 700, 3200, 200, 1700, 11000, 4100, 10000, 25300],
 'askvolume': [6900,
  9300,
  2500,
  3100,
  13200,
  72800,
  205400,
  66900,
  80700,
  214500],
 'last_price': 49.88,
 'trade_volume': 5851500,
 'trade_value': 291068717,
 'twap': 49.72945983546706}

## 时间分析

In [14]:
def timer_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Execution time of {func.__name__}: {end_time - start_time} seconds")
        return result
    return wrapper

@timer_decorator
def convert_api_to_df_format(api_response):
    # 创建一个字典来存储转换后的数据
    base_data = {
        'StockID': [],
        'AskPrice1': [], 'AskPrice2': [], 'AskPrice3': [], 'AskPrice4': [], 'AskPrice5': [],
        'AskPrice6': [], 'AskPrice7': [], 'AskPrice8': [], 'AskPrice9': [], 'AskPrice10': [],
        
        'AskVolume1': [], 'AskVolume2': [], 'AskVolume3': [], 'AskVolume4': [], 'AskVolume5': [],
        'AskVolume6': [], 'AskVolume7': [], 'AskVolume8': [], 'AskVolume9': [], 'AskVolume10': [],
        
        'BidPrice1': [], 'BidPrice2': [], 'BidPrice3': [], 'BidPrice4': [], 'BidPrice5': [],
        'BidPrice6': [], 'BidPrice7': [], 'BidPrice8': [], 'BidPrice9': [], 'BidPrice10': [],
        
        'BidVolume1': [], 'BidVolume2': [], 'BidVolume3': [], 'BidVolume4': [], 'BidVolume5': [],
        'BidVolume6': [], 'BidVolume7': [], 'BidVolume8': [], 'BidVolume9': [], 'BidVolume10': [],
        
        'TotalTradeVolume': [], 'TotalTradeValue': [],
        'localtime': [], 'limit_up_price': [],
        'limit_down_price': [],'last_price': [],
        'twap': []
    }

    for idx, item in enumerate(api_response):
        # 填充Ask价格和数量
        for i in range(10):
            base_data[f'AskPrice{i+1}'].append(item['askprice'][i])
            base_data[f'AskVolume{i+1}'].append(item['askvolume'][i])

        # 填充Bid价格和数量
        for i in range(10):
            base_data[f'BidPrice{i+1}'].append(item['bidprice'][i])
            base_data[f'BidVolume{i+1}'].append(item['bidvolume'][i])

        # 填充其他数据
        base_data['StockID'].append(f"UBIQ{idx:03}")
        base_data['TotalTradeVolume'].append(item['trade_volume'])
        base_data['TotalTradeValue'].append(item['trade_value'])
        base_data['localtime'].append(item['localtime'])
        base_data['limit_up_price'].append(item['limit_up_price'])
        base_data['limit_down_price'].append(item['limit_down_price'])
        base_data['twap'].append(item['twap'])
        base_data['last_price'].append(item['last_price'])

    # 创建DataFrame
    df = pd.DataFrame(base_data)
    return df

In [ ]:
a = [1,3]

In [ ]:
api.sendGetUserInfo(token_ub)['rows']

2024-08-27 00:25:28,559 [DEBUG] GetUserInfo: 


[{'instrument_name': 'UBIQ000',
  'share_holding': 0,
  'orders': 0,
  'error_orders': 0,
  'order_value': 0,
  'trade_value': 0,
  'target_volume': 11100,
  'remain_volume': 11100,
  'frozen_volume': 0},
 {'instrument_name': 'UBIQ001',
  'share_holding': 0,
  'orders': 0,
  'error_orders': 0,
  'order_value': 0,
  'trade_value': 0,
  'target_volume': 6000,
  'remain_volume': 6000,
  'frozen_volume': 0},
 {'instrument_name': 'UBIQ002',
  'share_holding': 0,
  'orders': 0,
  'error_orders': 0,
  'order_value': 0,
  'trade_value': 0,
  'target_volume': -8400,
  'remain_volume': -8400,
  'frozen_volume': 0},
 {'instrument_name': 'UBIQ003',
  'share_holding': 0,
  'orders': 0,
  'error_orders': 0,
  'order_value': 0,
  'trade_value': 0,
  'target_volume': -6000,
  'remain_volume': -6000,
  'frozen_volume': 0},
 {'instrument_name': 'UBIQ004',
  'share_holding': 0,
  'orders': 0,
  'error_orders': 0,
  'order_value': 0,
  'trade_value': 0,
  'target_volume': -12000,
  'remain_volume': -12000

In [ ]:
lobs

,StockID,AskPrice1,AskPrice2,AskPrice3,AskPrice4,AskPrice5,AskPrice6,AskPrice7,AskPrice8,AskPrice9,...,BidVolume8,BidVolume9,BidVolume10,TotalTradeVolume,TotalTradeValue,localtime,limit_up_price,limit_down_price,last_price,twap
0,UBIQ000,49.64,49.65,49.66,49.67,49.68,49.69,49.70,49.71,49.72,...,33500,34500,21500,15266000,755409313,1998,61.61875,36.97125,49.63,49.466939
1,UBIQ001,6.28,6.29,6.30,6.31,6.32,6.33,6.34,6.35,6.36,...,169700,336100,648300,64356800,402602051,1998,7.75625,4.65375,6.27,6.237489
2,UBIQ002,460.00,460.01,460.02,460.04,460.05,460.06,460.08,460.10,460.11,...,300,900,100,607700,279535024,1998,575.06250,345.03750,460.00,460.000219
3,UBIQ003,238.68,238.84,238.85,238.86,238.87,238.88,238.89,238.90,238.91,...,1700,400,600,795000,190097582,1998,299.24375,179.54625,238.68,239.079642
4,UBIQ004,17.16,17.17,17.18,17.19,17.20,17.21,17.22,17.23,17.24,...,84700,65600,133500,7295800,124959950,1998,21.40625,12.84375,17.15,17.128239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28995,UBIQ045,103.88,103.89,103.90,103.92,103.93,103.94,103.95,103.96,103.97,...,5100,400,700,1995500,207070215,2999,129.48750,77.69250,103.88,103.735082
28996,UBIQ046,19.02,19.04,19.05,19.06,19.07,19.08,19.09,19.10,19.11,...,21800,35900,11100,19409600,367851953,2999,23.09375,13.85625,19.01,18.949663
28997,UBIQ047,17.34,17.35,17.36,17.37,17.38,17.39,17.40,17.41,17.42,...,74100,121400,133200,13681800,236957178,2999,21.65625,12.99375,17.33,17.318634
28998,UBIQ048,52.13,52.14,52.15,52.16,52.17,52.18,52.19,52.20,52.21,...,1100,900,26400,4620500,240693850,2999,65.11875,39.07125,52.13,52.082225


In [ ]:
response

{'response_type': 'get_all_limit_order_book_response',
 'lobs': [{'localtime': 2340,
   'limit_up_price': 66.43125,
   'limit_down_price': 39.85875,
   'bidprice': [53.35,
    53.33,
    53.32,
    53.31,
    53.3,
    53.29,
    53.28,
    53.27,
    53.26,
    53.25],
   'askprice': [53.36,
    53.37,
    53.38,
    53.39,
    53.4,
    53.41,
    53.42,
    53.43,
    53.44,
    53.45],
   'bidvolume': [1300, 2600, 400, 10100, 3700, 12400, 6300, 2900, 34300, 100],
   'askvolume': [400, 4400, 29100, 7200, 10800, 12300, 5000, 8500, 9500, 9100],
   'last_price': 53.35,
   'trade_volume': 4136000,
   'trade_value': 219946999,
   'twap': 53.19598548080534},
  {'localtime': 2340,
   'limit_up_price': 23.05625,
   'limit_down_price': 13.83375,
   'bidprice': [18.74,
    18.73,
    18.72,
    18.71,
    18.7,
    18.69,
    18.68,
    18.67,
    18.66,
    18.65],
   'askprice': [18.75,
    18.76,
    18.77,
    18.78,
    18.79,
    18.8,
    18.81,
    18.82,
    18.83,
    18.84],
   'bi

In [ ]:
response['next_game_start_time'] + response['next_game_running_days'] * response['next_game_running_time']

1724664382

In [ ]:
api.sendGetUserInfo(token_ub)

2024-08-26 15:22:34,405 [DEBUG] GetUserInfo: 


{'response_type': 'get_user_info_response',
 'orders': 0,
 'error_orders': 0,
 'order_value': 0,
 'trade_value': 0,
 'rows': [{'instrument_name': 'UBIQ000',
   'share_holding': 0,
   'orders': 0,
   'error_orders': 0,
   'order_value': 0,
   'trade_value': 0,
   'target_volume': 5500,
   'remain_volume': 5500,
   'frozen_volume': 0},
  {'instrument_name': 'UBIQ001',
   'share_holding': 0,
   'orders': 0,
   'error_orders': 0,
   'order_value': 0,
   'trade_value': 0,
   'target_volume': 8700,
   'remain_volume': 8700,
   'frozen_volume': 0},
  {'instrument_name': 'UBIQ002',
   'share_holding': 0,
   'orders': 0,
   'error_orders': 0,
   'order_value': 0,
   'trade_value': 0,
   'target_volume': 4600,
   'remain_volume': 4600,
   'frozen_volume': 0},
  {'instrument_name': 'UBIQ003',
   'share_holding': 0,
   'orders': 0,
   'error_orders': 0,
   'order_value': 0,
   'trade_value': 0,
   'target_volume': 8300,
   'remain_volume': 8300,
   'frozen_volume': 0},
  {'instrument_name': 'UBIQ0

In [108]:
%%time
pd.DataFrame(api.sendGetAllLimitOrderBooks(token_ub)['lobs'])

CPU times: user 2.28 ms, sys: 1.22 ms, total: 3.5 ms
Wall time: 37 ms


,localtime,limit_up_price,limit_down_price,bidprice,askprice,bidvolume,askvolume,last_price,trade_volume,trade_value,twap
0,2380,63.55625,38.13375,"[50.73, 50.71, 50.7, 50.69, 50.68, 50.67, 50.6...","[50.74, 50.75, 50.76, 50.77, 50.79, 50.8, 50.8...","[1200, 9200, 14600, 5600, 79900, 21500, 22600,...","[1200, 6200, 1100, 900, 111000, 2000, 1500, 19...",50.74,8552100,435303407,50.874251
1,2380,9.48125,5.68875,"[7.42, 7.41, 7.4, 7.390000000000001, 7.38, 7.3...","[7.43, 7.44, 7.45, 7.46, 7.47, 7.48, 7.49, 7.5...","[158000, 479000, 371700, 252100, 237500, 23270...","[43200, 41300, 245000, 72300, 89400, 21900, 68...",7.43,27868900,208526015,7.484088
2,2380,425.71250,255.42750,"[342.91, 342.89, 342.87, 342.86, 342.85, 342.8...","[342.93, 342.94, 342.95, 342.96, 342.97, 342.9...","[100, 100, 1200, 600, 200, 600, 1100, 400, 400...","[500, 500, 800, 400, 100, 800, 500, 1300, 4900...",342.93,2205600,753886935,341.789346
3,2380,319.13125,191.47875,"[256.82, 256.78, 256.73, 256.68, 256.67, 256.6...","[256.84, 256.86, 256.87, 256.88, 256.89, 256.9...","[1500, 100, 100, 200, 400, 600, 300, 200, 300,...","[21200, 1400, 2200, 3400, 2600, 1500, 3800, 15...",256.82,2766100,709372309,256.401592
4,2380,21.91875,13.15125,"[17.22, 17.21, 17.2, 17.19, 17.18, 17.17, 17.1...","[17.23, 17.24, 17.25, 17.26, 17.27, 17.28, 17....","[409700, 47800, 99100, 135400, 68700, 140900, ...","[300, 3300, 28900, 500, 400, 79700, 10200, 282...",17.23,21417700,372841506,17.443968
5,2380,70.33750,42.20250,"[55.52, 55.51, 55.5, 55.49, 55.48, 55.47, 55.4...","[55.53, 55.54, 55.55, 55.56, 55.57, 55.58, 55....","[600, 7500, 16700, 300, 2800, 2200, 2500, 1310...","[100, 7400, 14000, 9300, 15200, 1600, 900, 130...",55.53,4659900,260125685,55.848937
6,2380,22.16875,13.30125,"[17.65, 17.64, 17.63, 17.62, 17.61, 17.6, 17.5...","[17.66, 17.67, 17.68, 17.69, 17.7, 17.71, 17.7...","[600, 18600, 16000, 12000, 19000, 87100, 83700...","[15400, 9300, 10000, 46000, 66400, 2700, 1600,...",17.65,8801100,155374247,17.654024
7,2380,78.96875,47.38125,"[62.87, 62.86, 62.85, 62.84, 62.83, 62.82, 62....","[62.88, 62.89, 62.9, 62.91, 62.92, 62.93, 62.9...","[3000, 8500, 9100, 400, 5300, 4000, 4500, 3700...","[2900, 8000, 4700, 2300, 7100, 1700, 600, 4800...",62.88,4838400,304823169,62.996213
8,2380,319.12500,191.47500,"[254.87, 254.84, 254.82, 254.81, 254.8, 254.76...","[254.89, 254.96, 254.97, 255, 255.03, 255.04, ...","[1700, 100, 700, 400, 900, 8300, 500, 200, 200...","[5600, 300, 100, 1200, 400, 100, 700, 100, 500...",254.89,1027900,261945197,254.754211
9,2380,15.30625,9.18375,"[12.02, 12.01, 12, 11.99, 11.98, 11.97, 11.96,...","[12.03, 12.04, 12.05, 12.06, 12.07, 12.08, 12....","[15300, 60600, 217200, 54600, 43600, 15700, 30...","[10600, 6900, 30400, 17900, 15600, 25000, 1600...",12.03,7093600,85841357,12.104507


In [ ]:
api.sendGetUserInfo(token_ub)['rows']

2024-08-25 16:25:45,228 [DEBUG] GetUserInfo: 


{'instrument_name': 'UBIQ000',
 'share_holding': 0,
 'orders': 0,
 'error_orders': 0,
 'order_value': 0,
 'trade_value': 0,
 'target_volume': 1800,
 'remain_volume': 1800,
 'frozen_volume': 0}

In [ ]:
# api.sendGetActiveOrder()
# api.sendGetInstrumentInfo()
# api.sendGetUserInfo(token_ub)
# api.sendGetTrade(token_ub, 'UBIQ000')
# api.sendGetGameInfo(token_ub)
api.sendGetAllLimitOrderBooks(token_ub)['lobs']

2024-08-23 19:02:30,395 [DEBUG] GetAllLimitOrderBooks


In [ ]:
api.sendGetUserInfo(token_ub)['rows'][1]

2024-08-25 19:20:08,612 [DEBUG] GetUserInfo: 
2024-08-25 19:20:08,612 [DEBUG] GetUserInfo: 


{'instrument_name': 'UBIQ001',
 'share_holding': -200,
 'orders': 4,
 'error_orders': 0,
 'order_value': 22772,
 'trade_value': 1055,
 'target_volume': -8700,
 'remain_volume': -8500,
 'frozen_volume': -4100}

In [ ]:
price = 0.1234
f'{price:.2}'

'0.12'

In [ ]:
pd.DataFrame(api.sendGetUserInfo(token_ub)['rows'])

2024-08-27 15:27:46,479 [DEBUG] GetUserInfo: 
2024-08-27 15:27:46,479 [DEBUG] GetUserInfo: 


,instrument_name,share_holding,orders,error_orders,order_value,trade_value,target_volume,remain_volume,frozen_volume
0,UBIQ000,0,0,11,0,0,6000,6000,0
1,UBIQ001,0,0,19,0,0,6000,6000,0
2,UBIQ002,0,0,18,0,0,6000,6000,0
3,UBIQ003,0,0,9,0,0,8700,8700,0
4,UBIQ004,0,0,12,0,0,6000,6000,0
5,UBIQ005,0,0,17,0,0,6000,6000,0
6,UBIQ006,0,0,22,0,0,-75000,-75000,0
7,UBIQ007,-7800,1,26,428532,429289,-7800,0,0
8,UBIQ008,6000,1,18,1230900,1230440,6000,0,0
9,UBIQ009,0,0,24,0,0,-57900,-57900,0


In [ ]:
{'token_ub': 'UBIQ_TEAM179_ANfgOwr3SvpN', 'user_info': 'NULL', 'instrument': 'UBIQ000', 'localtime': 18.047339916229248, 'direction': 'buy', 'price': 100.0, 'volume': 100}

{'token_ub': 'UBIQ_TEAM179_ANfgOwr3SvpN',
 'user_info': 'NULL',
 'instrument': 'UBIQ032',
 'localtime': 18.047339916229248,
 'direction': 'buy',
 'price': 7.0,
 'volume': 100}

In [ ]:
 {'token_ub': 'UBIQ_TEAM179_ANfgOwr3SvpN', 'user_info': '', 'instrument': 'UBIQ001', 'localtime': 1851.0175681114197, 'direction': 'sell', 'price': 5.12, 'volume': 3400.0}

{'token_ub': 'UBIQ_TEAM179_ANfgOwr3SvpN',
 'user_info': '',
 'instrument': 'UBIQ001',
 'localtime': 1851.0175681114197,
 'direction': 'sell',
 'price': 5.12,
 'volume': 3400.0}

In [ ]:
api.sendOrder(token_ub)

NameError: name 'api' is not defined

In [ ]:
api.sendGetActiveOrder(token_ub)

2024-08-25 16:27:10,572 [DEBUG] GetActiveOrder: 


{'response_type': 'get_active_order_response',
 'instruments': [{'instrument_name': 'UBIQ000', 'active_orders': []},
  {'instrument_name': 'UBIQ001', 'active_orders': []},
  {'instrument_name': 'UBIQ002', 'active_orders': []},
  {'instrument_name': 'UBIQ003', 'active_orders': []},
  {'instrument_name': 'UBIQ004', 'active_orders': []},
  {'instrument_name': 'UBIQ005', 'active_orders': []},
  {'instrument_name': 'UBIQ006', 'active_orders': []},
  {'instrument_name': 'UBIQ007', 'active_orders': []},
  {'instrument_name': 'UBIQ008', 'active_orders': []},
  {'instrument_name': 'UBIQ009', 'active_orders': []},
  {'instrument_name': 'UBIQ010', 'active_orders': []},
  {'instrument_name': 'UBIQ011', 'active_orders': []},
  {'instrument_name': 'UBIQ012', 'active_orders': []},
  {'instrument_name': 'UBIQ013', 'active_orders': []},
  {'instrument_name': 'UBIQ014', 'active_orders': []},
  {'instrument_name': 'UBIQ015', 'active_orders': []},
  {'instrument_name': 'UBIQ016', 'active_orders': []},
  {

In [ ]:
data = api.sendGetAllLimitOrderBooks(token_ub)
data['lobs']

2024-08-23 11:16:41,002 [DEBUG] GetAllLimitOrderBooks


In [ ]:
def ConvertToSimTime_us(start_time, time_ratio, day, running_time):
    return (time.time() - start_time - (day - 1) * running_time) * time_ratio

In [ ]:
round(0.5)

0

In [ ]:
print(bot.start_time)
print(bot.time_ratio)
print(bot.day)
print(bot.running_time)

1723698695
10
0
360


In [ ]:
bot = BotsDemoClass(username, password, 30020)
bot.login()
bot.init()
SimTimeLen = 3000 # 60*5 * 10
endWaitTime = 600 # 60*1 * 10
while True:
    if ConvertToSimTime_us(bot.start_time, bot.time_ratio, bot.day, bot.running_time) < SimTimeLen:
        break
    else:
        bot.day += 1


2024-08-23 20:47:09,130 [INFO] Login Success: UBIQ_TEAM179_ANfgOwr3SvpN
2024-08-23 20:47:09,131 [DEBUG] GetGameInfo: 
2024-08-23 20:47:09,153 [DEBUG] GetInstrumentInfo: 
2024-08-23 20:47:09,178 [INFO] Get Instruments: ['UBIQ000', 'UBIQ001', 'UBIQ002', 'UBIQ003', 'UBIQ004', 'UBIQ005', 'UBIQ006', 'UBIQ007', 'UBIQ008', 'UBIQ009', 'UBIQ010', 'UBIQ011', 'UBIQ012', 'UBIQ013', 'UBIQ014', 'UBIQ015', 'UBIQ016', 'UBIQ017', 'UBIQ018', 'UBIQ019', 'UBIQ020', 'UBIQ021', 'UBIQ022', 'UBIQ023', 'UBIQ024', 'UBIQ025', 'UBIQ026', 'UBIQ027', 'UBIQ028', 'UBIQ029', 'UBIQ030', 'UBIQ031', 'UBIQ032', 'UBIQ033', 'UBIQ034', 'UBIQ035', 'UBIQ036', 'UBIQ037', 'UBIQ038', 'UBIQ039', 'UBIQ040', 'UBIQ041', 'UBIQ042', 'UBIQ043', 'UBIQ044', 'UBIQ045', 'UBIQ046', 'UBIQ047', 'UBIQ048', 'UBIQ049']


{'response_type': 'get_game_info_response', 'next_game_start_time': 1724403057, 'next_game_running_days': 80, 'next_game_running_time': 360, 'next_game_time_ratio': 10, 'status': 'Success'}
{'response_type': 'get_instrument_info_response', 'instrument_number': 50, 'instruments': [{'id': 1, 'instrument_name': 'UBIQ000'}, {'id': 2, 'instrument_name': 'UBIQ001'}, {'id': 3, 'instrument_name': 'UBIQ002'}, {'id': 4, 'instrument_name': 'UBIQ003'}, {'id': 5, 'instrument_name': 'UBIQ004'}, {'id': 6, 'instrument_name': 'UBIQ005'}, {'id': 7, 'instrument_name': 'UBIQ006'}, {'id': 8, 'instrument_name': 'UBIQ007'}, {'id': 9, 'instrument_name': 'UBIQ008'}, {'id': 10, 'instrument_name': 'UBIQ009'}, {'id': 11, 'instrument_name': 'UBIQ010'}, {'id': 12, 'instrument_name': 'UBIQ011'}, {'id': 13, 'instrument_name': 'UBIQ012'}, {'id': 14, 'instrument_name': 'UBIQ013'}, {'id': 15, 'instrument_name': 'UBIQ014'}, {'id': 16, 'instrument_name': 'UBIQ015'}, {'id': 17, 'instrument_name': 'UBIQ016'}, {'id': 18, 'in

In [ ]:
def ConvertToSimTime_us(start_time, time_ratio, day, running_time):
    return (time.time() - start_time - (day - 1) * running_time) * time_ratio

while True:
    if ConvertToSimTime_us(bot.start_time, bot.time_ratio, bot.day, bot.running_time) < SimTimeLen:
        break
    else:
        bot.day += 1

In [ ]:
logger.setLevel(logging.INFO)
pres = 0
while bot.day <= bot.running_days:
    while True:
        if ConvertToSimTime_us(bot.start_time, bot.time_ratio, bot.day, bot.running_time) > -900:
            break
    bot.bod()
    now = round(ConvertToSimTime_us(bot.start_time, bot.time_ratio, bot.day, bot.running_time))
    for s in range(now, SimTimeLen + endWaitTime):
        while True:
            if ConvertToSimTime_us(bot.start_time, bot.time_ratio, bot.day, bot.running_time) >= s:
                break
        t = ConvertToSimTime_us(bot.start_time, bot.time_ratio, bot.day, bot.running_time)
        logger.info("Work Time: {} {}".format(s, t))
        if t < SimTimeLen - 30:
            bot.work()
    bot.eod()
    bot.day += 1
bot.final()

logger.setLevel(logging.DEBUG)

2024-08-23 20:50:42,542 [INFO] Work Time: 3455 3455.419981479645
2024-08-23 20:50:42,600 [INFO] Work Time: 3456 3456.0000109672546
2024-08-23 20:50:42,700 [INFO] Work Time: 3457 3457.000000476837
2024-08-23 20:50:42,800 [INFO] Work Time: 3458 3458.000009059906
2024-08-23 20:50:42,900 [INFO] Work Time: 3459 3459.000289440155
2024-08-23 20:50:43,000 [INFO] Work Time: 3460 3460.0
2024-08-23 20:50:43,100 [INFO] Work Time: 3461 3461.0000109672546
2024-08-23 20:50:43,200 [INFO] Work Time: 3462 3462.000000476837
2024-08-23 20:50:43,300 [INFO] Work Time: 3463 3463.000009059906
2024-08-23 20:50:43,400 [INFO] Work Time: 3464 3464.0000009536743
2024-08-23 20:50:43,500 [INFO] Work Time: 3465 3465.0
2024-08-23 20:50:43,600 [INFO] Work Time: 3466 3466.0000109672546
2024-08-23 20:50:43,700 [INFO] Work Time: 3467 3467.000000476837
2024-08-23 20:50:43,800 [INFO] Work Time: 3468 3468.000009059906
2024-08-23 20:50:43,900 [INFO] Work Time: 3469 3469.0000009536743
2024-08-23 20:50:44,000 [INFO] Work Time: 

KeyboardInterrupt: 

# 数据分析

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
example_df = pd.read_csv('./snapshots/0_UBIQ000.csv', sep='|')

In [ ]:
window = example_df.iloc[4:14]

In [ ]:
window['BidPrice1']

4     64.71
5     64.71
6     64.72
7     64.71
8     64.69
9     64.69
10    64.73
11    64.73
12    64.74
13    64.74
Name: BidPrice1, dtype: float64

In [ ]:
window['BidPrice10']

4     64.62
5     64.62
6     64.63
7     64.62
8     64.60
9     64.60
10    64.62
11    64.62
12    64.64
13    64.64
Name: BidPrice10, dtype: float64

In [ ]:
window['TotalTradeVolume'].pct_change()

4     18100
5     21000
6     21600
7     23300
8     49700
9     50700
10    52500
11    53800
12    56800
13    57800
Name: TotalTradeVolume, dtype: int64

In [ ]:
example_df['AskVolume1']

0       1200
1        200
2       4700
3        200
4       3300
        ... 
2995    1500
2996     800
2997    1100
2998    1100
2999     500
Name: AskVolume1, Length: 3000, dtype: int64

In [ ]:
def load_and_process_data(file_path):
    df = pd.read_csv(file_path, sep='|')
    df['TotalVolume'] = df['AskVolume1'] + df['AskVolume2'] + df['AskVolume3'] + df['AskVolume4'] + df['AskVolume5'] + \
                        df['BidVolume1'] + df['BidVolume2'] + df['BidVolume3'] + df['BidVolume4'] + df['BidVolume5']
    return df

def calculate_liquidity_metrics(data):
    daily_volume = data['TotalTradeVolume'].sum()
    volume_volatility = data['TotalTradeVolume'].std()
    volume_distribution = stats.describe(data['TotalTradeVolume'])
    
    return {
        'daily_volume': daily_volume,
        'volume_volatility': volume_volatility,
        'volume_distribution': volume_distribution
    }

def analyze_stocks(folder_path):
    results = {}
    for file in os.listdir(folder_path):
        if file.endswith('.csv'):
            stock_id = file.split('_')[1].split('.')[0]
            data = load_and_process_data(os.path.join(folder_path, file))
            metrics = calculate_liquidity_metrics(data)
            results[stock_id] = metrics
    
    return results

def calculate_target_ratios(results, target_volume):
    ratios = {}
    for stock_id, metrics in results.items():
        mean_volume = metrics['daily_volume']
        std_volume = metrics['volume_volatility']
        
        ratio_mean = abs(target_volume) / mean_volume if mean_volume != 0 else 0
        ratio_std = abs(target_volume) / std_volume if std_volume != 0 else 0
        
        ratios[stock_id] = {
            'ratio_to_mean': min(ratio_mean, 10),
            'ratio_to_std': min(ratio_std, 10)
        }
    
    return ratios

def generate_report(results, ratios):
    report = "股票流动性分析报告\n\n"
    
    for stock_id in results.keys():
        report += f"股票 {stock_id}:\n"
        report += f"  日均成交量: {results[stock_id]['daily_volume']:.2f}\n"
        report += f"  成交量波动性: {results[stock_id]['volume_volatility']:.2f}\n"
        report += f"  成交量分布:\n"
        report += f"    最小值: {results[stock_id]['volume_distribution'].minmax[0]:.2f}\n"
        report += f"    最大值: {results[stock_id]['volume_distribution'].minmax[1]:.2f}\n"
        report += f"    均值: {results[stock_id]['volume_distribution'].mean:.2f}\n"
        report += f"    方差: {results[stock_id]['volume_distribution'].variance:.2f}\n"
        report += f"    偏度: {results[stock_id]['volume_distribution'].skewness:.2f}\n"
        report += f"    峰度: {results[stock_id]['volume_distribution'].kurtosis:.2f}\n"
        report += f"  目标仓位/日均成交量比率: {ratios[stock_id]['ratio_to_mean']:.2f}\n"
        report += f"  目标仓位/成交量标准差比率: {ratios[stock_id]['ratio_to_std']:.2f}\n\n"
    
    return report

def plot_volume_distribution(results):
    fig, axs = plt.subplots(5, 3, figsize=(20, 30))
    fig.suptitle('股票成交量分布')
    
    for i, (stock_id, metrics) in enumerate(results.items()):
        row = i // 3
        col = i % 3
        
        volume_data = metrics['volume_distribution']
        axs[row, col].hist(volume_data, bins=30, edgecolor='black')
        axs[row, col].set_title(f'股票 {stock_id}')
        axs[row, col].set_xlabel('成交量')
        axs[row, col].set_ylabel('频率')
    
    plt.tight_layout()
    plt.savefig('volume_distribution.png')

def main():
    folder_path = 'snapshots'
    target_volume = 10000  # 假设的目标仓位
    
    results = analyze_stocks(folder_path)
    ratios = calculate_target_ratios(results, target_volume)
    
    report = generate_report(results, ratios)
    print(report)
    
    with open('liquidity_report.txt', 'w') as f:
        f.write(report)
    
    plot_volume_distribution(results)

In [ ]:
def analyze_liquidity(data, target_positions):
    """
    分析股票流动性并评估目标仓位相对于历史成交量的规模。

    :param data: DataFrame，包含每只股票的历史成交量数据
    :param target_positions: dict，键为股票代码，值为目标仓位
    :return: DataFrame，包含每只股票的流动性分析结果
    """
    results = []
    
    for stock in data.columns:
        stock_data = data[stock].dropna()
        
        # 计算日均成交量和标准差
        mean_volume = stock_data.mean()
        std_volume = stock_data.std()
        
        # 获取目标仓位
        target_position = abs(target_positions.get(stock, 0))
        
        # 计算比率
        ratio_to_mean = target_position / mean_volume if mean_volume != 0 else np.inf
        ratio_to_std = target_position / std_volume if std_volume != 0 else np.inf
        
        # 将比率映射到0-10的范围
        scaled_ratio_mean = min(ratio_to_mean * 10, 10)
        scaled_ratio_std = min(ratio_to_std * 10, 10)
        
        results.append({
            'stock': stock,
            'mean_daily_volume': mean_volume,
            'volume_std': std_volume,
            'target_position': target_position,
            'ratio_to_mean_volume': ratio_to_mean,
            'ratio_to_volume_std': ratio_to_std,
            'scaled_ratio_mean': scaled_ratio_mean,
            'scaled_ratio_std': scaled_ratio_std
        })
    
    results_df = pd.DataFrame(results)
    
    # 添加描述性统计
    stats = results_df[['ratio_to_mean_volume', 'ratio_to_volume_std', 'scaled_ratio_mean', 'scaled_ratio_std']].describe()
    
    return results_df, stats

# 数据预处理

## 源数据获取

In [48]:
import time 
import pandas as pd

def ConvertToSimTime_us(start_time, time_ratio, day, running_time):
    return (time.time() - start_time - (day - 1) * running_time) * time_ratio

def convert_LOB_response_to_df_format(api_response):
    # 创建一个字典来存储转换后的数据
    base_data = {
        'Tick': [], 'StockID': [],
        'AskPrice1': [], 'AskPrice2': [], 'AskPrice3': [], 'AskPrice4': [], 'AskPrice5': [],
        'AskPrice6': [], 'AskPrice7': [], 'AskPrice8': [], 'AskPrice9': [], 'AskPrice10': [],
        
        'AskVolume1': [], 'AskVolume2': [], 'AskVolume3': [], 'AskVolume4': [], 'AskVolume5': [],
        'AskVolume6': [], 'AskVolume7': [], 'AskVolume8': [], 'AskVolume9': [], 'AskVolume10': [],
        
        'BidPrice1': [], 'BidPrice2': [], 'BidPrice3': [], 'BidPrice4': [], 'BidPrice5': [],
        'BidPrice6': [], 'BidPrice7': [], 'BidPrice8': [], 'BidPrice9': [], 'BidPrice10': [],
        
        'BidVolume1': [], 'BidVolume2': [], 'BidVolume3': [], 'BidVolume4': [], 'BidVolume5': [],
        'BidVolume6': [], 'BidVolume7': [], 'BidVolume8': [], 'BidVolume9': [], 'BidVolume10': [],
        
        'TotalTradeVolume': [], 'TotalTradeValue': [],
        'limit_up_price': [],
        'limit_down_price': [],'last_price': [],
        'twap': []
    }

    for idx, item in enumerate(api_response):
        # 填充Ask价格和数量
        for i in range(10):
            base_data[f'AskPrice{i+1}'].append(item['askprice'][i])
            base_data[f'AskVolume{i+1}'].append(item['askvolume'][i])

        # 填充Bid价格和数量
        for i in range(10):
            base_data[f'BidPrice{i+1}'].append(item['bidprice'][i])
            base_data[f'BidVolume{i+1}'].append(item['bidvolume'][i])

        # 填充总成交量和总成交额
        base_data['TotalTradeVolume'].append(item['trade_volume'])
        base_data['TotalTradeValue'].append(item['trade_value'])
        base_data['Tick'].append(item['localtime'])
        base_data['StockID'].append(f'UBIQ{idx:03}')
        
        base_data['limit_up_price'].append(item['limit_up_price'])
        base_data['limit_down_price'].append(item['limit_down_price'])
        base_data['twap'].append(item['twap'])
        base_data['last_price'].append(item['last_price'])

    # 创建DataFrame
    df = pd.DataFrame(base_data)
    return df

def convert_userinfo_response_to_df_format(api_response, t):
    # 创建一个字典来存储转换后的数据
    base_data = {
        'Tick': [], 'StockID': [],
        'share_holding': [],
        'orders': [],
        'error_orders': [], 'order_value': [],
        'trade_value': [],
        'target_volume': [],'remain_volume': [],
        'frozen_volume': []
    }

    for idx, item in enumerate(api_response):
        # 填充总成交量和总成交额
        base_data['Tick'].append(t)
        base_data['StockID'].append(f'UBIQ{idx:03}')
        
        base_data['share_holding'].append(item['share_holding'])
        base_data['orders'].append(item['orders'])
        base_data['error_orders'].append(item['error_orders'])
        base_data['order_value'].append(item['order_value'])
        base_data['trade_value'].append(item['trade_value'])
        base_data['target_volume'].append(item['target_volume'])
        base_data['remain_volume'].append(item['remain_volume'])
        base_data['frozen_volume'].append(item['frozen_volume'])

    # 创建DataFrame
    df = pd.DataFrame(base_data)
    return df

2024-08-28 15:13:24,664 [DEBUG] GetUserInfo: 


In [72]:
uf_df

,Tick,StockID,share_holding,orders,error_orders,order_value,trade_value,target_volume,remain_volume,frozen_volume
0,1,UBIQ000,0,0,0,0,0,11700,11700,0
1,1,UBIQ001,-3800,6,0,25553,26022,-8400,-4600,0
2,1,UBIQ002,0,0,0,0,0,6000,6000,0
3,1,UBIQ003,-8400,11,0,2176982,2199386,-12000,-3600,0
4,1,UBIQ004,1700,3,0,30286,30146,14100,12400,0
5,1,UBIQ005,-500,3,0,26436,26717,-8100,-7600,0
6,1,UBIQ006,0,0,0,0,0,34800,34800,0
7,1,UBIQ007,3500,4,0,190369,188780,8400,4900,0
8,1,UBIQ008,0,0,0,0,0,6000,6000,0
9,1,UBIQ009,-1700,3,0,27194,27536,-25200,-23500,0


In [59]:
pd.read_csv('./data_collection/snapshots/20240828-001043-day32_all_stocks.csv').columns

Index(['Tick', 'StockID', 'AskPrice1', 'AskPrice2', 'AskPrice3', 'AskPrice4',
       'AskPrice5', 'AskPrice6', 'AskPrice7', 'AskPrice8', 'AskPrice9',
       'AskPrice10', 'AskVolume1', 'AskVolume2', 'AskVolume3', 'AskVolume4',
       'AskVolume5', 'AskVolume6', 'AskVolume7', 'AskVolume8', 'AskVolume9',
       'AskVolume10', 'BidPrice1', 'BidPrice2', 'BidPrice3', 'BidPrice4',
       'BidPrice5', 'BidPrice6', 'BidPrice7', 'BidPrice8', 'BidPrice9',
       'BidPrice10', 'BidVolume1', 'BidVolume2', 'BidVolume3', 'BidVolume4',
       'BidVolume5', 'BidVolume6', 'BidVolume7', 'BidVolume8', 'BidVolume9',
       'BidVolume10', 'TotalTradeVolume', 'TotalTradeValue', 'RemainPositions',
       'TargetPositions'],
      dtype='object')

In [ ]:
while(True):
    response = api.sendGetAllLimitOrderBooks(token_ub)['lobs']
    # print(response)
    if bs._cache_lobs is None or response[0]['localtime'] > bs._cache_lobs['Tick'].max():
        bs._union_lobs(bs.convert_api_to_df_format(response))

KeyboardInterrupt: 

In [182]:
df = pd.read_csv('./price_series.csv')

In [47]:
df

,Tick,StockID,AskPrice1,AskPrice2,AskPrice3,AskPrice4,AskPrice5,AskPrice6,AskPrice7,AskPrice8,...,BidVolume7,BidVolume8,BidVolume9,BidVolume10,TotalTradeVolume,TotalTradeValue,limit_up_price,limit_down_price,last_price,twap
0,2557,UBIQ000,50.94,50.95,50.96,50.97,50.98,50.99,51.00,51.02,...,9500,17000,11600,10800,8147100,417225081,63.84375,38.30625,50.93,51.203167
1,2557,UBIQ001,7.49,7.50,7.51,7.52,7.53,7.54,7.55,7.56,...,395400,494300,427200,719900,120411400,901105539,9.26875,5.56125,7.48,7.485497
2,2557,UBIQ002,457.40,457.41,457.42,457.43,457.44,457.45,457.46,457.47,...,800,700,100,400,1043300,476894154,573.26250,343.95750,457.40,457.191871
3,2557,UBIQ003,235.06,235.08,235.09,235.10,235.11,235.12,235.13,235.14,...,6300,3800,4100,2700,5198800,1229267373,298.00625,178.80375,235.06,236.288529
4,2557,UBIQ004,17.03,17.04,17.05,17.06,17.07,17.08,17.09,17.10,...,130500,95800,102100,134500,14788600,252839016,21.48125,12.88875,17.02,17.098377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,2590,UBIQ045,116.80,116.81,116.82,116.85,116.86,116.87,116.88,116.89,...,200,300,3600,300,4870400,573514665,149.03750,89.42250,116.80,117.825972
1696,2590,UBIQ046,19.01,19.02,19.03,19.04,19.05,19.06,19.07,19.08,...,72600,106300,100300,198500,60744600,1152919529,23.70625,14.22375,19.00,18.970192
1697,2590,UBIQ047,17.20,17.21,17.22,17.23,17.24,17.25,17.26,17.27,...,270000,205600,177300,827600,18551600,319284287,21.55625,12.93375,17.20,17.216163
1698,2590,UBIQ048,52.33,52.34,52.35,52.36,52.37,52.38,52.41,52.42,...,100,8000,300,100,9385100,494416972,65.20625,39.12375,52.32,52.573914


In [ ]:
bs._cache_lobs['BidPrice1']

0        50.93
1         7.48
2       457.36
3       235.05
4        17.02
         ...  
1695    116.79
1696     19.00
1697     17.19
1698     52.32
1699     62.60
Name: BidPrice1, Length: 1700, dtype: float64

In [ ]:
bs._cache_lobs.to_csv('./temp_raw_data.csv', index=False)

In [16]:
df = pd.read_csv('./temp_raw_data.csv')

In [22]:
a = df.iloc[:50]

In [29]:
a[a['StockID'] == 'UBIQ000']['AskPrice1'].iloc[0]

50.94

In [19]:
df[df['StockID'] == 'UBIQ000']['AskPrice1']

0       50.94
50      50.95
100     50.94
150     50.95
200     50.95
250     50.95
300     50.96
350     50.97
400     50.96
450     50.97
500     50.97
550     50.97
600     50.97
650     50.97
700     50.98
750     50.98
800     50.98
850     50.98
900     50.97
950     50.97
1000    50.98
1050    50.99
1100    50.99
1150    50.99
1200    50.99
1250    50.98
1300    50.99
1350    50.96
1400    50.98
1450    50.98
1500    50.98
1550    50.98
1600    50.99
1650    50.99
Name: AskPrice1, dtype: float64

In [ ]:
period_df = df.groupby('StockID').tail(20)

In [62]:
api.sendGetGameInfo(token_ub)

2024-08-28 16:00:03,186 [DEBUG] GetGameInfo: 


{'response_type': 'get_game_info_response',
 'next_game_start_time': 1724810272,
 'next_game_running_days': 80,
 'next_game_running_time': 360,
 'next_game_time_ratio': 10,
 'status': 'Success'}

In [ ]:
period_df

,Tick,StockID,AskPrice1,AskPrice2,AskPrice3,AskPrice4,AskPrice5,AskPrice6,AskPrice7,AskPrice8,...,BidVolume7,BidVolume8,BidVolume9,BidVolume10,TotalTradeVolume,TotalTradeValue,limit_up_price,limit_down_price,last_price,twap
700,2571,UBIQ000,50.98,50.99,51.00,51.02,51.03,51.04,51.05,51.06,...,25800,23300,15200,11400,8214900,420680148,63.84375,38.30625,50.98,51.201863
701,2571,UBIQ001,7.49,7.50,7.51,7.52,7.53,7.54,7.55,7.56,...,394400,494300,427200,733500,120502500,901787639,9.26875,5.56125,7.49,7.485507
702,2571,UBIQ002,457.42,457.43,457.44,457.45,457.46,457.47,457.48,457.49,...,800,100,100,100,1046900,478540744,573.26250,343.95750,457.41,457.193033
703,2571,UBIQ003,235.07,235.08,235.09,235.10,235.11,235.12,235.13,235.14,...,19700,6300,4100,4100,5224700,1235355290,298.00625,178.80375,235.07,236.281183
704,2571,UBIQ004,17.03,17.04,17.05,17.06,17.07,17.08,17.09,17.10,...,130500,94800,102100,135100,14833400,253601702,21.48125,12.88875,17.02,17.097984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,2590,UBIQ045,116.80,116.81,116.82,116.85,116.86,116.87,116.88,116.89,...,200,300,3600,300,4870400,573514665,149.03750,89.42250,116.80,117.825972
1696,2590,UBIQ046,19.01,19.02,19.03,19.04,19.05,19.06,19.07,19.08,...,72600,106300,100300,198500,60744600,1152919529,23.70625,14.22375,19.00,18.970192
1697,2590,UBIQ047,17.20,17.21,17.22,17.23,17.24,17.25,17.26,17.27,...,270000,205600,177300,827600,18551600,319284287,21.55625,12.93375,17.20,17.216163
1698,2590,UBIQ048,52.33,52.34,52.35,52.36,52.37,52.38,52.41,52.42,...,100,8000,300,100,9385100,494416972,65.20625,39.12375,52.32,52.573914


## 因子分析

In [187]:
def calculate_factors(df : pd.DataFrame, window_short=5, window_medium=10, window_long=30):
    # logger.info(f'type of df: {type(df)}')
    factors = pd.DataFrame(index=df.index)
    factors['Tick'] = df['Tick']
    factors['StockID'] = df['StockID']
    
    # 假设我们使用 last_price 作为价格序列
    price_series = df['last_price']
    
    # 价格动量
    factors['momentum_short'] = price_series.pct_change(window_short)
    factors['momentum_medium'] = price_series.pct_change(window_medium)
    factors['momentum_long'] = price_series.pct_change(window_long)
    
    # 移动平均
    factors['ma_short'] = price_series.rolling(window=window_short).mean()
    factors['ma_medium'] = price_series.rolling(window=window_medium).mean()
    factors['ma_long'] = price_series.rolling(window=window_long).mean()
    
    # 移动平均交叉
    factors['ma_cross_short_medium'] = factors['ma_short'] / factors['ma_medium'] - 1
    factors['ma_cross_short_long'] = factors['ma_short'] / factors['ma_long'] - 1
    factors['ma_cross_medium_long'] = factors['ma_medium'] / factors['ma_long'] - 1
    
    # 波动率
    factors['volatility_short'] = price_series.rolling(window=window_short).std()
    factors['volatility_medium'] = price_series.rolling(window=window_medium).std()
    factors['volatility_long'] = price_series.rolling(window=window_long).std()
    
    # 相对强弱指标 (RSI)
    delta = price_series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window_medium).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window_medium).mean()
    rs = gain / loss
    factors['rsi'] = 100 - (100 / (1 + rs))
    
    # 布林带
    rolling_mean = price_series.rolling(window=window_medium).mean()
    rolling_std = price_series.rolling(window=window_medium).std()
    factors['bollinger_upper'] = rolling_mean + (rolling_std * 2)
    factors['bollinger_lower'] = rolling_mean - (rolling_std * 2)
    factors['bollinger_percent'] = (price_series - factors['bollinger_lower']) / (factors['bollinger_upper'] - factors['bollinger_lower'])
    
    # 价格加速度
    factors['price_acceleration'] = factors['momentum_short'].diff()
    
    # 成交量相关因子 (假设使用 TotalTradeVolume)
    volume_series = df['TotalTradeVolume']
    factors['volume_momentum'] = volume_series.pct_change(window_short)
    factors['volume_ma_ratio'] = volume_series / volume_series.rolling(window=window_medium).mean()
    
    # 价量相关性
    factors['price_volume_corr'] = price_series.rolling(window=window_medium).corr(volume_series)
    
    # 资金流量指标 (MFI)，假设你有 high_series 和 low_series (使用 BidPrice1 和 AskPrice1)
    high_series = df['BidPrice1']
    low_series = df['AskPrice1']
    typical_price = (price_series + high_series + low_series) / 3
    raw_money_flow = typical_price * volume_series
    positive_flow = raw_money_flow.where(typical_price > typical_price.shift(1), 0).rolling(window=window_medium).sum()
    negative_flow = raw_money_flow.where(typical_price < typical_price.shift(1), 0).rolling(window=window_medium).sum()
    mfi_ratio = positive_flow / negative_flow
    factors['mfi'] = 100 - (100 / (1 + mfi_ratio))
    
    # 真实波幅 (ATR)
    high_low = high_series - low_series
    high_close = np.abs(high_series - price_series.shift())
    low_close = np.abs(low_series - price_series.shift())
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    factors['atr'] = true_range.rolling(window=window_medium).mean()
    
    # 价格通道
    factors['channel_upper'] = high_series.rolling(window=window_medium).max()
    factors['channel_lower'] = low_series.rolling(window=window_medium).min()
    factors['channel_position'] = (price_series - factors['channel_lower']) / (factors['channel_upper'] - factors['channel_lower'])
    
    # 趋势强度指标
    def calculate_trend_strength(series, window):
        slopes = [linregress(range(window), series.iloc[i:i+window])[0] for i in range(len(series) - window + 1)]
        return pd.Series(slopes + [np.nan] * (window - 1), index=series.index)
    
    factors['trend_strength'] = calculate_trend_strength(price_series, window_medium)
    
    expected_factors  = [
        'Tick', 'StockID',
        'momentum_short',
        'momentum_medium',
        'momentum_long',
        'ma_short',
        'ma_medium',
        'ma_long',
        'ma_cross_short_medium',
        'ma_cross_short_long',
        'ma_cross_medium_long',
        'volatility_short',
        'volatility_medium',
        'volatility_long',
        'rsi',
        'bollinger_upper',
        'bollinger_lower',
        'bollinger_percent',
        'price_acceleration',
        'volume_momentum',
        'volume_ma_ratio',
        'price_volume_corr',
        'mfi',
        'atr',
        'channel_upper',
        'channel_lower',
        'channel_position',
        'trend_strength'
    ]
    
    for factor in expected_factors:
        if factor not in factors.columns:
            factors[factor] = 0  # 或者使用其他合适的默认值

    factors = factors.replace([np.inf, -np.inf], np.nan).fillna(0)

    return factors[expected_factors]  # 只返回预期的因子


In [188]:
%%time
calculate_factors(df)

CPU times: user 118 ms, sys: 12.1 ms, total: 130 ms
Wall time: 166 ms


,Tick,StockID,momentum_short,momentum_medium,momentum_long,ma_short,ma_medium,ma_long,ma_cross_short_medium,ma_cross_short_long,...,price_acceleration,volume_momentum,volume_ma_ratio,price_volume_corr,mfi,atr,channel_upper,channel_lower,channel_position,trend_strength
0,2557,UBIQ000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.00,0.000000,-6.689879
1,2557,UBIQ001,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.00,0.000000,7.272545
2,2557,UBIQ002,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.00,0.000000,5.260061
3,2557,UBIQ003,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.00,0.000000,16.492061
4,2557,UBIQ004,0.000000,0.000000,0.000000,153.578,0.000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.00,0.000000,15.037030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,2590,UBIQ045,10.715145,1.280359,-0.617425,143.548,102.838,93.427667,0.395865,0.536461,...,2.722008,-0.887502,0.125119,-0.436826,57.332932,142.118,458.48,5.08,0.246405,0.000000
1696,2590,UBIQ046,0.077708,2.747535,-0.863614,143.822,104.231,89.417333,0.379839,0.608435,...,-10.637437,0.808338,2.331386,-0.558159,55.811829,147.283,458.48,9.61,0.020919,0.000000
1697,2590,UBIQ047,-0.767505,-0.926809,-0.123790,132.466,82.451,89.336333,0.606603,0.482779,...,-0.845213,0.405499,0.679036,-0.490196,43.089827,124.465,458.48,9.61,0.016909,0.000000
1698,2590,UBIQ048,0.032360,4.450000,1.687211,132.794,86.723,90.431333,0.531243,0.468451,...,0.799865,0.715897,0.467352,-0.485758,53.526744,105.438,458.48,9.98,0.094404,0.000000


In [ ]:
import torch
from scipy.stats import linregress
from sklearn.preprocessing import StandardScaler

def preprocess_dataframe(df, sequence_length=10, forward_periods=10):
    # 按StockID和Tick排序
    df = df.sort_values(['StockID', 'Tick'])
    
    # 计算中间价格
    df['MiddlePrice'] = (df['BidPrice1'] * df['AskVolume1'] + df['AskPrice1'] * df['BidVolume1']) / (df['AskVolume1'] + df['BidVolume1'])
    
    # 初始化结果列表
    all_sequences = []
    all_labels = []
    all_stock_ids = []

    # 对每个股票分别处理
    for stock_id, stock_data in df.groupby('StockID'):
        price_series = stock_data['MiddlePrice']
        volume_series = stock_data['TotalTradeVolume']
        high_series = stock_data['AskPrice1']
        low_series = stock_data['BidPrice1']

        # 计算因子
        factors = calculate_factors(price_series, volume_series, high_series, low_series)

        if len(factors) < sequence_length + forward_periods:
            continue  # 跳过数据不足的股票

        # 标准化因子
        scaler = StandardScaler()
        scaled_factors = scaler.fit_transform(factors)

        # 计算未来收益率
        returns = price_series.pct_change(periods=forward_periods).shift(-forward_periods)

        for i in range(len(scaled_factors) - sequence_length - forward_periods + 1):
            sequence = scaled_factors[i:i+sequence_length]
            label = returns.iloc[i+sequence_length-1]

            if not np.isnan(label):
                all_sequences.append(sequence)
                all_labels.append(label)
                all_stock_ids.append(stock_id)

    if not all_sequences:
        return None, None, None  # 如果没有有效序列，返回None

    # 转换为PyTorch张量
    X = torch.tensor(np.array(all_sequences), dtype=torch.float32)
    y = torch.tensor(np.array(all_labels), dtype=torch.float32)
    stock_ids = np.array(all_stock_ids)

    return X, y, stock_ids

def calculate_factors(price_series, volume_series=None, high_series=None, low_series=None, window_short=5, window_medium=10, window_long=30):
    factors = pd.DataFrame(index=price_series.index)
    
    # 价格动量
    factors['momentum_short'] = price_series.pct_change(window_short)
    factors['momentum_medium'] = price_series.pct_change(window_medium)
    factors['momentum_long'] = price_series.pct_change(window_long)
    
    # 移动平均
    factors['ma_short'] = price_series.rolling(window=window_short).mean()
    factors['ma_medium'] = price_series.rolling(window=window_medium).mean()
    factors['ma_long'] = price_series.rolling(window=window_long).mean()
    
    # 移动平均交叉
    factors['ma_cross_short_medium'] = factors['ma_short'] / factors['ma_medium'] - 1
    factors['ma_cross_short_long'] = factors['ma_short'] / factors['ma_long'] - 1
    factors['ma_cross_medium_long'] = factors['ma_medium'] / factors['ma_long'] - 1
    
    # 波动率
    factors['volatility_short'] = price_series.rolling(window=window_short).std()
    factors['volatility_medium'] = price_series.rolling(window=window_medium).std()
    factors['volatility_long'] = price_series.rolling(window=window_long).std()
    
    # 相对强弱指标 (RSI)
    delta = price_series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window_medium).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window_medium).mean()
    rs = gain / loss
    factors['rsi'] = 100 - (100 / (1 + rs))
    
    # 布林带
    rolling_mean = price_series.rolling(window=window_medium).mean()
    rolling_std = price_series.rolling(window=window_medium).std()
    factors['bollinger_upper'] = rolling_mean + (rolling_std * 2)
    factors['bollinger_lower'] = rolling_mean - (rolling_std * 2)
    factors['bollinger_percent'] = (price_series - factors['bollinger_lower']) / (factors['bollinger_upper'] - factors['bollinger_lower'])
    
    # 价格加速度
    factors['price_acceleration'] = factors['momentum_short'].diff()
    
    if volume_series is not None:
        # 成交量相关因子
        factors['volume_momentum'] = volume_series.pct_change(window_short)
        factors['volume_ma_ratio'] = volume_series / volume_series.rolling(window=window_medium).mean()
        
        # 价量相关性
        factors['price_volume_corr'] = price_series.rolling(window=window_medium).corr(volume_series)
        
        # 资金流量指标 (MFI)
        if high_series is not None and low_series is not None:
            typical_price = (price_series + high_series + low_series) / 3
            raw_money_flow = typical_price * volume_series
            positive_flow = raw_money_flow.where(typical_price > typical_price.shift(1), 0).rolling(window=window_medium).sum()
            negative_flow = raw_money_flow.where(typical_price < typical_price.shift(1), 0).rolling(window=window_medium).sum()
            mfi_ratio = positive_flow / negative_flow
            factors['mfi'] = 100 - (100 / (1 + mfi_ratio))
    
    if high_series is not None and low_series is not None:
        # 真实波幅 (ATR)
        high_low = high_series - low_series
        high_close = np.abs(high_series - price_series.shift())
        low_close = np.abs(low_series - price_series.shift())
        true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
        factors['atr'] = true_range.rolling(window=window_medium).mean()
        
        # 价格通道
        factors['channel_upper'] = high_series.rolling(window=window_medium).max()
        factors['channel_lower'] = low_series.rolling(window=window_medium).min()
        factors['channel_position'] = (price_series - factors['channel_lower']) / (factors['channel_upper'] - factors['channel_lower'])
    
    # 趋势强度指标
    def calculate_trend_strength(series, window):
        slopes = [linregress(range(window), series.iloc[i:i+window])[0] for i in range(len(series) - window + 1)]
        return pd.Series(slopes + [np.nan] * (window - 1), index=series.index)
    
    factors['trend_strength'] = calculate_trend_strength(price_series, window_medium)
    
    # 删除包含 NaN 的行
    factors = factors.replace([np.inf, -np.inf], np.nan).fillna(0)
    
    return factors


In [ ]:
%%time
api.sendGetAllLimitOrderBooks(token_ub)

NameError: name 'api' is not defined

In [ ]:
%%time
# print(period_df.shape)
# start_time = time.time()
X, y ,stock_ids = preprocess_dataframe(period_df)
# end_time = time.time()
# print(f"Preprocessing time: {end_time - start_time} seconds")

CPU times: user 461 ms, sys: 12.1 ms, total: 473 ms
Wall time: 515 ms


In [ ]:
period_df[(
        (period_df['StockID'] == "UBIQ000") &
        (period_df['Tick'] == 2571)
    )]

,Tick,StockID,AskPrice1,AskPrice2,AskPrice3,AskPrice4,AskPrice5,AskPrice6,AskPrice7,AskPrice8,...,BidVolume8,BidVolume9,BidVolume10,TotalTradeVolume,TotalTradeValue,limit_up_price,limit_down_price,last_price,twap,StockIndex
700,2571,UBIQ000,50.98,50.99,51.0,51.02,51.03,51.04,51.05,51.06,...,23300,15200,11400,8214900,420680148,63.84375,38.30625,50.98,51.201863,0


In [ ]:
def factorDF2Tensor(df: pd.DataFrame):
    r"""
    把pd的表格形式的数字转化为tensor的形式
    要求df有StockID,Tick列，其余均为特征列
    
    """
    assert df.shape[0] % 50 == 0
    
    tick_num, feature_num = df.shape[0] // 50, df.shape[1]
    temp_df = df.sort_values(['StockID', 'Tick'])
    features = temp_df.columns.drop(['StockID', 'Tick'])
    
    # 将数据转换为PyTorch tensor
    tensor_data = torch.tensor(temp_df[features].values, dtype=torch.float32)
    
    # 重塑tensor为(50, tick_num, 46)的形状 (46是特征数，不包括StockID和Tick)
    reshaped_tensor = tensor_data.view(50, tick_num, -1)
    return reshaped_tensor

In [ ]:
%%time

factorDF2Tensor(period_df)

CPU times: user 2.1 ms, sys: 2.18 ms, total: 4.28 ms
Wall time: 5.73 ms


tensor([[[ 50.9800,  50.9900,  51.0000,  ...,  50.9800,  51.2019,   0.0000],
         [ 50.9800,  50.9900,  51.0000,  ...,  50.9600,  51.2018,   0.0000],
         [ 50.9800,  50.9900,  51.0000,  ...,  50.9600,  51.2017,   0.0000],
         ...,
         [ 50.9800,  50.9900,  51.0000,  ...,  50.9700,  51.2005,   0.0000],
         [ 50.9900,  51.0000,  51.0200,  ...,  50.9800,  51.2004,   0.0000],
         [ 50.9900,  51.0000,  51.0200,  ...,  50.9900,  51.2003,   0.0000]],

        [[  7.4900,   7.5000,   7.5100,  ...,   7.4900,   7.4855,   1.0000],
         [  7.4900,   7.5000,   7.5100,  ...,   7.4800,   7.4855,   1.0000],
         [  7.4800,   7.4900,   7.5000,  ...,   7.4800,   7.4855,   1.0000],
         ...,
         [  7.4900,   7.5000,   7.5100,  ...,   7.4900,   7.4855,   1.0000],
         [  7.4900,   7.5000,   7.5100,  ...,   7.4800,   7.4855,   1.0000],
         [  7.4900,   7.5000,   7.5100,  ...,   7.4900,   7.4855,   1.0000]],

        [[457.4200, 457.4300, 457.4400,  ...

In [ ]:
period_df.dtypes

Tick                  int64
StockID              object
AskPrice1           float64
AskPrice2           float64
AskPrice3           float64
AskPrice4           float64
AskPrice5           float64
AskPrice6           float64
AskPrice7           float64
AskPrice8           float64
AskPrice9           float64
AskPrice10          float64
AskVolume1            int64
AskVolume2            int64
AskVolume3            int64
AskVolume4            int64
AskVolume5            int64
AskVolume6            int64
AskVolume7            int64
AskVolume8            int64
AskVolume9            int64
AskVolume10           int64
BidPrice1           float64
BidPrice2           float64
BidPrice3           float64
BidPrice4           float64
BidPrice5           float64
BidPrice6           float64
BidPrice7           float64
BidPrice8           float64
BidPrice9           float64
BidPrice10          float64
BidVolume1            int64
BidVolume2            int64
BidVolume3            int64
BidVolume4          

In [ ]:
print(X.shape)
print(y.shape)

torch.Size([4800, 10, 26])
torch.Size([4800])


In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

def calculate_factors(df, sequence_length=10):
    # 计算中间价
    mid_price = (df['AskPrice1'] + df['BidPrice1']) / 2
    
    # 计算价格动量
    price_momentum_short = mid_price.pct_change(5)
    price_momentum_mid = mid_price.pct_change(30)
    
    # 计算交易量变化
    volume_change_short = df['TotalTradeVolume'].pct_change(5)
    volume_change_mid = df['TotalTradeVolume'].pct_change(30)
    
    # 计算价差
    spread = (df['AskPrice1'] - df['BidPrice1']) / mid_price
    
    # 计算订单簿不平衡
    depth_imbalance = (df['BidVolume1'] - df['AskVolume1']) / (df['BidVolume1'] + df['AskVolume1'])
    
    # 计算波动率
    volatility = mid_price.rolling(30).std() / mid_price
    
    # 计算RSI
    delta = mid_price.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    
    # 计算移动平均线交叉
    ma_short = mid_price.rolling(window=5).mean()
    ma_long = mid_price.rolling(window=30).mean()
    ma_cross = ma_short / ma_long - 1
    
    # 将所有因子组合在一起
    factors = pd.concat([
        price_momentum_short,
        price_momentum_mid,
        volume_change_short,
        volume_change_mid,
        spread,
        depth_imbalance,
        volatility,
        rsi,
        ma_cross
    ], axis=1)
    
    factors.columns = ['price_momentum_short', 'price_momentum_mid', 'volume_change_short', 'volume_change_mid', 
                       'spread', 'depth_imbalance', 'volatility', 'rsi', 'ma_cross']
    factors = factors.replace([np.inf, -np.inf], np.nan).fillna(0)

    return factors

def load_and_preprocess_data(directory, sequence_length=10):
    all_data = []
    all_labels = []
    
    for filename in tqdm(os.listdir(directory), desc="Loading files"):
        if filename.endswith(".csv"):
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath, delimiter='|')
            
            factors = calculate_factors(df, sequence_length)
            factors = factors.dropna()
            
            scaler = StandardScaler()
            scaled_factors = scaler.fit_transform(factors)
            
            for i in range(len(scaled_factors) - sequence_length):
                sequence = scaled_factors[i:i+sequence_length]
                label = df['BidPrice1'].iloc[i+sequence_length] - df['BidPrice1'].iloc[i+sequence_length-1]
                all_data.append(sequence)
                all_labels.append(label)
    
    return torch.tensor(all_data, dtype=torch.float32), torch.tensor(all_labels, dtype=torch.float32)

def save_model(model, filename):
    torch.save(model.state_dict(), filename)

def load_model(filename, input_dim, hidden_dim, num_layers, output_dim):
    model = LSTMModel(input_dim, hidden_dim, num_layers, output_dim)
    model.load_state_dict(torch.load(filename))
    return model

def train_lstm_model(X, y, hidden_dim=32, num_layers=2, test_size=0.2, random_state=42, batch_size=64, num_epochs=1, gpu_id=0):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    print(X_train.size)
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    if torch.cuda.is_available():
        device = torch.device(f"cuda:{gpu_id}")
        print(f"Using GPU: {torch.cuda.get_device_name(device)}")
    else:
        device = torch.device("cpu")
        print("CUDA is not available. Using CPU.")
    
    input_dim = X.shape[2]  # number of features
    output_dim = 1  # predicting a single value
    
    model = LSTMModel(input_dim, hidden_dim, num_layers, output_dim).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters())
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch_X, batch_y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs.squeeze(), batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")
    
        # model.eval()
        # with torch.no_grad():
        #     X_test = X_test.to(device)
        #     y_pred = model(X_test).cpu()
        #     rmse = torch.sqrt(criterion(y_pred.squeeze(), y_test))
        #     print(f"RMSE: {rmse.item():.4f}")
    
    return model


In [ ]:
directory = './snapshots_raw'
sequence_length = 10

X, y = load_and_preprocess_data(directory, sequence_length)

Loading files: 100%|██████████| 3000/3000 [01:53<00:00, 26.32it/s]
/var/folders/7v/0pft7_9d053gl9jmqh7tn0sr0000gn/T/ipykernel_69219/529460355.py:100: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1712608659634/work/torch/csrc/utils/tensor_new.cpp:277.)
  return torch.tensor(all_data, dtype=torch.float32), torch.tensor(all_labels, dtype=torch.float32)


In [ ]:
X.shape

torch.Size([8714444, 10, 9])